In [1]:
import os
import json
import openai
from datetime import datetime, timedelta
#from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
from amadeus import Client, ResponseError

In [2]:
# load_dotenv()
# openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
# amadeus = Client(
#     client_id='H4S446vjDRHVJn8C0ZDRkXSLt03AvOGp',
#     client_secret='2MCfJEBzkSLqYla0',
#     #log_level='debug'

# )


In [6]:
client = OpenAI()

In [7]:
def get_airport_code(location):
    """Get the airport code for a given location string."""
    try:
        response = amadeus.reference_data.locations.get(
            keyword=location,
            subType='AIRPORT'
        )
        if response.data:
            # Return the first matching airport code
            return response.data[0]['iataCode']
        else:
            print(f"No airport code found for {location}")
            return None
    except ResponseError as error:
        print(f"Error finding airport code for {location}: {error}")
        return None

# Flight info API

In [8]:
def extract_flight_info_from_query(query):
    today = datetime.now()
    current_date_str = today.strftime('%Y-%m-%d')
    messages = [
        {
            "role": "system",
            "content": (
                "You are an assistant that helps extract flight information from user queries. "
                f"Today is {current_date_str}. Extract the following details from the query: "
                "1. location_origin: The departure city or airport, ensure it corresponds to a valid airport code "
                "2. location_destination: The destination city or airport, ensure it corresponds to a valid airport code "
                "3. departure_date: The date of departure "
                "4. adults: The number of adult passengers. (If there is no information regarding this then consider only 1 Adult is there)"
                "If the query specifies a relative date (e.g., 'next Monday'), convert it to an absolute date. "
                "Provide the information in JSON format as follows: "
                '{"location_origin": "origin", "location_destination": "destination", "departure_date": "YYYY-MM-DD", "adults": number_of_adults}'
            )
        },
        {
            "role": "user",
            "content": query
        }
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=300,
        temperature=0.1 
    )

    response_text = response.choices[0].message.content.strip()
    try:
        flight_info = json.loads(response_text)
        # Basic validation
        required_keys = ["location_origin", "location_destination", "departure_date", "adults"]
        if not all(key in flight_info for key in required_keys):
            raise ValueError("Incomplete response from LLM")
        return flight_info
    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error extracting flight info: {e}")
        return None


In [9]:
def get_flight_info(location_origin, location_destination, departure_date, adults=1):
    """Get flight information between two locations."""
    origin_code = location_origin
    destination_code = location_destination
    
    if not origin_code or not destination_code:
        return "Could not find airport codes for the provided locations."

    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=origin_code,
            destinationLocationCode=destination_code,
            departureDate=departure_date,
            adults=adults
        )
        return response.data
    except ResponseError as error:
        return str(error)

In [10]:
def create_flight_dataframe(flight_data):
    """Create a Pandas DataFrame from the flight data."""
    flight_details = []

    # Get the airline names from the flight offers
    airlines = set()
    for offer in flight_data:
        for segment in offer['itineraries'][0]['segments']:
            airlines.add(segment['carrierCode'])

    # Fetch airline names using the IATA codes
    airline_names = {}
    for airline_code in airlines:
        airline_response = amadeus.reference_data.airlines.get(airlineCodes=airline_code)
        if airline_response.data:
                airline_names[airline_code] = airline_response.data[0]['commonName']

    for flight in flight_data:
        total_price = flight['price'].get('total', '')
        currency = flight['price'].get('currency', '')
        one_way = len(flight['itineraries']) == 1
        
        for itinerary in flight['itineraries']:
            num_stops = len(itinerary['segments']) - 1
            
            for segment in itinerary['segments']:
                airline_code = segment.get('carrierCode', '')
                airline_name =  airline_names.get(segment['carrierCode'], '')
                from_ = segment.get(0, {}).get('departure', {}).get('iataCode', '')
                from_terminal = segment.get(0, {}).get('departure', {}).get('terminal', '')
                to = segment.get(0, {}).get('arrival', {}).get('iataCode', '')
                to_terminal = segment.get(0, {}).get('arrival', {}).get('terminal', '')
                departure = segment['departure'].get('at', '')
                arrival = segment['arrival'].get('at', '')
                cabin = segment.get('travelerPricings', [{}])[0].get('fareDetailsBySegment', [{}])[0].get('cabin', '')
                pricing_detail = segment.get('pricingDetailPerAdult', {})
               

                flight_details.append({
                    "Airline Code": airline_code,
                    "Airline Name": airline_name,
                    "Departure": departure,
                    "Arrival": arrival,
                    "Total Price": total_price,
                    "Currency": currency,
                    "Number of Stops": num_stops,
                    "Cabin": cabin,
                    "One Way": one_way})
                   
    data = pd.DataFrame(flight_details)
    data.drop_duplicates(inplace=True)
    return data

In [11]:
query = "Give me flight information for Bangalore to Mumbai for next Monday"
result = extract_flight_info_from_query(query)

In [12]:
result

{'location_origin': 'BLR',
 'location_destination': 'BOM',
 'departure_date': '2024-05-27',
 'adults': 1}

In [13]:
origin = result['location_origin']
destination = result['location_destination']
departure_date = result['departure_date']
adults = result['adults']

flight_info = get_flight_info(origin, destination, departure_date)

# Convert the flight info to a DataFrame
if isinstance(flight_info, list):
    flight_df = create_flight_dataframe(flight_info)
else:
    print("Error in forming Dataframe")

1. CHeapest One
2. Less Hops
3. Less travel time
4. User Currency (better to ask / follow up question)

In [14]:
flight_df

,Airline Code,Airline Name,Departure,Arrival,Total Price,Currency,Number of Stops,Cabin,One Way
0,UK,VISTARA,2024-05-27T08:30:00,2024-05-27T10:10:00,52.34,EUR,0,,True
1,UK,VISTARA,2024-05-27T06:40:00,2024-05-27T08:25:00,52.34,EUR,0,,True
2,AI,AIR INDIA,2024-05-27T06:40:00,2024-05-27T08:35:00,53.86,EUR,0,,True
3,AI,AIR INDIA,2024-05-27T21:25:00,2024-05-27T23:20:00,53.86,EUR,0,,True
4,AI,AIR INDIA,2024-05-27T09:20:00,2024-05-27T11:20:00,53.86,EUR,0,,True
...,...,...,...,...,...,...,...,...,...
111,H1,HAHN AIR SYSTEMS,2024-05-28T12:00:00,2024-05-28T14:25:00,440.85,EUR,1,,True
113,H1,HAHN AIR SYSTEMS,2024-05-27T18:20:00,2024-05-27T20:45:00,440.85,EUR,1,,True
115,H1,HAHN AIR SYSTEMS,2024-05-27T19:00:00,2024-05-27T21:25:00,440.85,EUR,1,,True
116,H1,HAHN AIR SYSTEMS,2024-05-27T05:40:00,2024-05-27T08:20:00,469.20,EUR,1,,True


# Using Agent

In [40]:
df1 = flight_df.copy()

In [54]:
def prompt_query(query,origin,destination):
    system_prompt = "System Prompt : " + f"You are an assistant that helps to answer queries based on the dataframe provided. This is a dataframe containing flight information from {origin} to {destination}. So answer the question by analyzing dataframe and give information useful for ticket booking only, please refrain from explaining how things are calculated. Don't give the code but analyze the dataframe, calculate values if required by yourself and answer the query."
    query_ask = "Query : " + query
    return f"{system_prompt}\n{query_ask}"


In [76]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

In [80]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df1, verbose=True)

In [93]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df1,
    verbose=False,
    handle_parsing_errors=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

c:\Users\rajpo\PycharmProjects\Tecnokrat\GenAI\venv\Lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:249: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [94]:
query = "Give me cheapest flight for Bangalore to Mumbai for next Monday"
query_to_ask = prompt_query(query,origin,destination)

In [95]:
res = agent.invoke(query_to_ask)
res['output'].strip()

TypeError: Completions.create() got an unexpected keyword argument 'functions'

https://python.langchain.com/v0.2/docs/how_to/sql_csv/

# Using Pandas Toolkit

In [17]:
df = flight_df.copy()

In [18]:
from langchain_experimental.agents import create_pandas_dataframe_agent

In [33]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0613")

In [58]:
def prompt_query(query,origin,destination):
    main_prompt = f"""
    "You are an assistant that helps to answer queries based on the flight information dataframe provided. 
    This is a dataframe containing flight information from {origin} to {destination}. 
    So answer the question by analyzing dataframe and give direct answers to query and please refrain from explaining how things are calculated. 
    Don't give the code but analyze the dataframe, calculate values if required by yourself and answer the query."

    """
    system_prompt = "System Prompt : " + main_prompt
    query_ask = "Query : " + query
    return f"{system_prompt}\n{query_ask}"


In [59]:
query = "give top 3 flights which are cheapest and also have less travel time"
query_to_ask = prompt_query(query,origin,destination)

In [60]:
print(query_to_ask)

System Prompt : 
    "You are an assistant that helps to answer queries based on the flight information dataframe provided. 
    This is a dataframe containing flight information from BLR to BOM. 
    So answer the question by analyzing dataframe and give direct answers to query and please refrain from explaining how things are calculated. 
    Don't give the code but analyze the dataframe, calculate values if required by yourself and answer the query."

    
Query : give top 3 flights which are cheapest and also have less travel time


In [61]:
agent = create_pandas_dataframe_agent(llm, df, agent_type="openai-tools", verbose=False)
res = agent.invoke(
    {
        "input": query_to_ask
    }
)

print(res['output'])

To find the top 3 flights that are cheapest and have less travel time, we can sort the dataframe by both "Total Price" and the time difference between "Arrival" and "Departure". 

Here is the answer:
1. Flight 2: Airline Name - AIR INDIA, Total Price - 53.86 EUR, Travel Time - 1 hour and 55 minutes
2. Flight 1: Airline Name - VISTARA, Total Price - 52.34 EUR, Travel Time - 1 hour and 45 minutes
3. Flight 0: Airline Name - VISTARA, Total Price - 52.34 EUR, Travel Time - 1 hour and 40 minutes
